## Preprocess Income Dataset 

### Import Libraries & Data 

In [100]:
# need to make script to scrape correspondence file
# https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/correspondences/CG_SA2_2016_SA2_2021.csv

In [101]:
import warnings
import regex as re
import pandas as pd

warnings.filterwarnings('ignore')

In [102]:
# Household Income 
income_2016 = pd.read_csv("../data/landing/2016Census_G29_VIC_SA2.csv")
income_2021 = pd.read_csv("../data/landing/2021Census_G33_VIC_SA2.csv")

# SA2 Codes and Suburb Names 
sa2_2016 = pd.read_csv("../data/landing/SA2_codes_2016.csv")
sa2_2021 = pd.read_excel("../data/landing/SA2_codes_2021.xlsx", sheet_name=0)

# correspondence = pd.read_csv('../data/landing/CG_SA2_2016_SA2_2021.csv')

### Preprocess Income Data

In [103]:
# Check Schema and Null Values 
# print(income_2016.info())
# print(income_2021.info())

# Make column names lowercase
income_2016.columns = income_2016.columns.str.lower()
income_2021.columns = income_2021.columns.str.lower()

# Make the column names cohesive
income_2016 = income_2016.rename(columns={
    "sa2_maincode_2016": "sa2_code_2016"
})

# Only retrieve the total household income columns 
income_2016 = income_2016[['sa2_code_2016'] + income_2016.filter(regex='_tot$').columns.tolist()]
income_2021 = income_2021[['sa2_code_2021'] + income_2021.filter(regex='_tot$').columns.tolist()]

# Drop the last 3 columns as they are not relevant 
income_2016 = income_2016.iloc[:, :-3]
income_2021 = income_2021.iloc[:, :-3]

# Drop the second column as it is not relevant 
income_2016 = income_2016.drop(income_2016.columns[1], axis=1)
income_2021 = income_2021.drop(income_2021.columns[1], axis=1)

### Preprocess the SA2 Codes and Suburb Names Dataframes

In [104]:
# Make column names lowercase
sa2_2016.columns = sa2_2016.columns.str.lower()
sa2_2021.columns = sa2_2021.columns.str.lower()

# Make all string columns lower case
sa2_2016 = sa2_2016.applymap(lambda x: x.lower() if isinstance(x, str) else x)
sa2_2021 = sa2_2021.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Only retrieve the SA2 codes and suburb names 
sa2_2016 = sa2_2016[["sa2_maincode_2016", "sa2_name_2016"]]
sa2_2021 = sa2_2021[["asgs_structure", "census_code_2021", "census_name_2021"]]

# For 2021, filter only for SA2 codes and names 
sa2_2021 = sa2_2021[
    (sa2_2021['asgs_structure'] == 'sa2') & 
    (sa2_2021['census_code_2021'].astype(str).str.startswith('2'))
]

# Clean the column names and dataframe 
sa2_2016 = sa2_2016.rename(columns={
    "sa2_maincode_2016": "sa2_code_2016"
})

sa2_2021 = sa2_2021.rename(columns={
    "census_code_2021": "sa2_code_2021",
    "census_name_2021": "sa2_name_2021"
}).drop(columns=["asgs_structure"])

# Exlcude the (vic.) that some rows have in sa2_name 
sa2_2016['sa2_name_2016'] = sa2_2016['sa2_name_2016'].str.replace(r'\s*\(vic\.\)', '', regex=True)
sa2_2021['sa2_name_2021'] = sa2_2021['sa2_name_2021'].str.replace(r'\s*\(vic\.\)', '', regex=True)

# Check Schema 
# print(sa2_2016.info())
# print(sa2_2021.info())

# Convert code columns to int64 and name columns to string 
sa2_2016['sa2_code_2016'] = sa2_2016['sa2_code_2016'].astype('int64')
sa2_2016['sa2_name_2016'] = sa2_2016['sa2_name_2016'].astype('str')

sa2_2021['sa2_code_2021'] = sa2_2021['sa2_code_2021'].astype('int64')
sa2_2021['sa2_name_2021'] = sa2_2021['sa2_name_2021'].astype('str')

### Map SA2 Codes in the Income Dataset to the Suburb Names

In [105]:
# Merge income datasets with sa2 datasets to replace the SA2 code with the SA2 name
income_2016 = income_2016.merge(sa2_2016, left_on='sa2_code_2016', right_on='sa2_code_2016', how='inner')
income_2016['sa2_code_2016'] = income_2016['sa2_name_2016']  
income_2016 = income_2016.drop(['sa2_code_2016'], axis=1)  

# Reorder to place sa2_name_2016 as the first column
income_2016 = income_2016[['sa2_name_2016'] + [col for col in income_2016.columns if col != 'sa2_name_2016']]

# Do the same for 2021 income dataset
income_2021 = income_2021.merge(sa2_2021, left_on='sa2_code_2021', right_on='sa2_code_2021', how='inner')
income_2021['sa2_code_2021'] = income_2021['sa2_name_2021']  
income_2021 = income_2021.drop(['sa2_code_2021'], axis=1)  

income_2021 = income_2021[['sa2_name_2021'] + [col for col in income_2021.columns if col != 'sa2_name_2021']]

### Correspondence

This is where I thought of doing the mapping for 2016 sa2 name to 2021 sa2 name

In [106]:
# correspondence = pd.read_csv('../data/landing/CG_SA2_2016_SA2_2021.csv')

# correspondence['SA2_NAME_2016'] = correspondence['SA2_NAME_2016'].str.lower()
# correspondence['SA2_NAME_2021'] = correspondence['SA2_NAME_2021'].str.lower()

# correspondence = correspondence[~correspondence['SA2_NAME_2016'].isna() & ~correspondence['SA2_NAME_2021'].isna()]

# # Only get data for Victoria
# correspondence = correspondence[correspondence['SA2_CODE_2021'].str.startswith('2')]

### Summing and Removing Directional Modifiers (Saleha's Preprocessing Code)

In [107]:
# Define directional modifiers and the word 'surrounds' to be removed
directional_modifiers = [' - east', ' - west', ' - north', ' - south', ' - central', ' surrounds', ' (north)', ' (south)', ' (east)', ' (west)', ' region']
pattern = '|'.join([re.escape(suffix) for suffix in directional_modifiers])

income_2016['sa2_name_2016'] = income_2016['sa2_name_2016'].str.replace(pattern, '', regex=True)
income_2021['sa2_name_2021'] = income_2021['sa2_name_2021'].str.replace(pattern, '', regex=True)

# Split sa2_name where multiple names are separated by hyphens
income_2016['sa2_name_2016'] = income_2016['sa2_name_2016'].str.split(' - ')
income_2021['sa2_name_2021'] = income_2021['sa2_name_2021'].str.split(' - ')

# Explode the lists into separate rows
income_2016_exploded = income_2016.explode('sa2_name_2016')
income_2016_exploded = income_2016_exploded.reset_index(drop=True)

income_2021_exploded = income_2021.explode('sa2_name_2021')
income_2021_exploded = income_2021_exploded.reset_index(drop=True)

# Remove trailing whitespaces
income_2016_exploded['sa2_name_2016'] = income_2016_exploded['sa2_name_2016'].str.rstrip()
income_2021_exploded['sa2_name_2021'] = income_2021_exploded['sa2_name_2021'].str.rstrip()

# Mapping for the SA2 names to the correct suburbs
sa2_name_mapping = {
    'ballarat' : 'ballarat central',
    'flemington racecourse' : 'flemington',
    'southbank wharf' : 'south wharf',
    'port melbourne industrial' : 'port melbourne',
    'reservoir east' : 'reservoir',
    'reservoir west' : 'reservoir',
    'research warrandyte' : 'warrandyte',
    'essendon airport' : 'essendon',
    'gladstone parkmeadows' : 'gladstone park',
    'craigieburn west' : 'craigieburn',
    'wandin' : 'wandin north',
    'pakenham east' : 'pakenham',
    'pakenham west' : 'pakenham',
    'narre warren west' : 'narre warren',
    'berwick east' : 'berwick',
    'berwick west' : 'berwick',
    'point cook east' : 'point cook',
    'point cook west' : 'point cook',
    'truganina east' : 'truganina',
    'truganina west' : 'truganina',
    'melbourne cbd' : 'melbourne'
}

# Remove the "(vic.)" from sa2_name values
income_2016_exploded['sa2_name_2016'] = income_2016_exploded['sa2_name_2016'].str.replace(r'\s*\(vic\.\)', '', regex=True)
income_2016_exploded['sa2_name_2016'] = income_2016_exploded['sa2_name_2016'].replace(sa2_name_mapping)

income_2021_exploded['sa2_name_2021'] = income_2021_exploded['sa2_name_2021'].str.replace(r'\s*\(vic\.\)', '', regex=True)
income_2021_exploded['sa2_name_2021'] = income_2021_exploded['sa2_name_2021'].replace(sa2_name_mapping)

# Convert counts to integers
tot_cols = income_2016.columns[income_2016.columns.str.contains('tot')]

income_2016_exploded[tot_cols] = income_2016_exploded[tot_cols].astype('int')
income_2021_exploded[tot_cols] = income_2021_exploded[tot_cols].astype('int')

# Create the aggregation dictionary
aggregation_functions = {col: 'sum' for col in tot_cols}

# Apply the groupby and aggregation
income_2016_grouped = income_2016_exploded.groupby('sa2_name_2016').agg(aggregation_functions).reset_index()
income_2021_grouped = income_2021_exploded.groupby('sa2_name_2021').agg(aggregation_functions).reset_index()

In [257]:
# List of suburbs that are in 2016 but not in 2021
# list(income_2021_grouped[~income_2021_grouped['suburb'].isin(income_2016_grouped['suburb'])]['suburb'].unique())

# Is exactly the same as the 3rd code chunk under Predictions

In [108]:
# Rename SA2 name column to be suburb
income_2016_grouped = income_2016_grouped.rename(columns={
    "sa2_name_2016": "suburb"
})

income_2021_grouped = income_2021_grouped.rename(columns={
    "sa2_name_2021": "suburb"
})

# Check dataframes that there are no suburb names with '-'
# print(income_2016_grouped[income_2016_grouped['suburb'].str.contains(r'-', na=False)])
# print(income_2021_grouped[income_2021_grouped['suburb'].str.contains(r'-', na=False)])

# Check both dataframes have the same column names 
print(income_2016_grouped.columns.equals(income_2021_grouped.columns))

# add a tag for the year to each column
income_2016_grouped = income_2016_grouped.rename(columns={col: f"{col}_2016" if col != 'suburb' else col for col in income_2016_grouped.columns})
income_2021_grouped = income_2021_grouped.rename(columns={col: f"{col}_2021" if col != 'suburb' else col for col in income_2021_grouped.columns})

# Check dataframe dimensions
print(income_2016_grouped.shape)
print(income_2021_grouped.shape)

# View final dataframes 
income_2021_grouped.head()

True
(478, 17)
(520, 17)


,suburb,hi_1_149_tot_2021,hi_150_299_tot_2021,hi_300_399_tot_2021,hi_400_499_tot_2021,hi_500_649_tot_2021,hi_650_799_tot_2021,hi_800_999_tot_2021,hi_1000_1249_tot_2021,hi_1250_1499_tot_2021,hi_1500_1749_tot_2021,hi_1750_1999_tot_2021,hi_2000_2499_tot_2021,hi_2500_2999_tot_2021,hi_3000_3499_tot_2021,hi_3500_3999_tot_2021,hi_4000_more_tot_2021
0,abbotsford,28,42,82,119,140,113,164,259,308,253,274,675,295,332,186,891
1,aberfeldie,35,62,102,250,175,246,271,363,384,342,286,638,397,413,280,1394
2,airport west,28,60,78,215,110,213,178,237,235,198,198,387,244,242,177,395
3,albert park,47,81,124,221,185,258,273,384,405,382,361,849,321,495,251,1995
4,albion,25,65,119,220,169,178,189,203,235,167,167,302,184,139,85,195


### Save Dataframes

In [113]:
income_2016_grouped.to_csv('../data/curated/income_2016.csv', index=False)
income_2021_grouped.to_csv('../data/curated/income_2021.csv', index=False)

print("All Saved!")

All Saved!


## Predictions

In [124]:
income_2016 = pd.read_csv('../data/curated/income_2016.csv')
income_2021 = pd.read_csv('../data/curated/income_2021.csv')

In [133]:
# Assuming the columns are structured similarly
df_2016 = income_2016.set_index('suburb')
df_2021 = income_2021.set_index('suburb')

# Rename columns for merging clarity
df_2016.columns = df_2016.columns.str.replace('_2016', '')
df_2021.columns = df_2021.columns.str.replace('_2021', '')

# Create a combined DataFrame for interpolation and prediction
df_combined = pd.concat([df_2016.assign(year=2016), df_2021.assign(year=2021)], axis=0).reset_index()

# Set multi-index with year and suburb
#df_combined = df_combined.set_index(['suburb', 'year'])

# Find suburbs that have 2021 data
suburbs_with_2021 = df_combined[df_combined['year'] == 2021]['suburb']

# Filter out rows where the suburb is not in the list of suburbs with 2021 data
df_combined = df_combined[df_combined['suburb'].isin(suburbs_with_2021)]

In [135]:
import numpy as np 

all_years = np.arange(2016, 2022)

# Ensure each suburb has rows for every year (including the missing years)
df_complete = (
    df_combined
    .groupby('suburb')
    .apply(lambda group: group.set_index('year').reindex(all_years).ffill().bfill().reset_index())
    .reset_index(drop=True)
)

columns_with_tot = [col for col in df_complete.columns if 'tot' in col]

# Set the values to NaN for the years 2017 to 2020 for these columns
df_complete.loc[df_complete['year'].between(2017, 2020), columns_with_tot] = np.nan

# df_complete.head(20)

In [136]:
# Fill in the missing values for 2017 - 2020
df_interpolated = df_complete.groupby('suburb').apply(lambda x: x.interpolate(method='linear')).reset_index(drop=True)

# Make them integers since they're counts
df_interpolated[columns_with_tot] = df_interpolated[columns_with_tot].astype(int)

In [138]:
df_interpolated[df_interpolated['suburb'] == 'melbourne']

,year,suburb,hi_1_149_tot,hi_150_299_tot,hi_300_399_tot,hi_400_499_tot,hi_500_649_tot,hi_650_799_tot,hi_800_999_tot,hi_1000_1249_tot,hi_1250_1499_tot,hi_1500_1749_tot,hi_1750_1999_tot,hi_2000_2499_tot,hi_2500_2999_tot,hi_3000_3499_tot,hi_3500_3999_tot,hi_4000_more_tot
1710,2016,melbourne,438,592,446,522,617,765,950,1268,1093,901,768,1352,567,363,541,626
1711,2017,melbourne,420,588,492,563,691,838,1061,1411,1242,1016,875,1584,635,468,514,788
1712,2018,melbourne,402,585,539,605,765,911,1173,1555,1391,1131,982,1817,703,574,488,951
1713,2019,melbourne,385,581,586,647,840,984,1284,1698,1540,1246,1090,2049,771,680,462,1114
1714,2020,melbourne,367,578,633,689,914,1057,1396,1842,1689,1361,1197,2282,839,786,436,1277
1715,2021,melbourne,350,575,680,731,989,1131,1508,1986,1839,1476,1305,2515,907,892,410,1440


In [269]:
# Note that for suburbs that did not exist in 2016, their 2017-2020 values == 2021 value
# For an example, try uncommenting the line below
# df_interpolated[df_interpolated['suburb'] == 'armstrong creek']

# TESTING NEW ARIMA

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

df = df_interpolated.copy()

# List of columns to forecast
columns_to_forecast = df.columns[2:]  # Assuming the first two columns are 'year' and 'suburb'

# Forecast years
forecast_years = [2022, 2023, 2024, 2025, 2026, 2027]

# Create an empty list to store forecasted results
forecast_results = []

# Iterate over each suburb
for suburb in df['suburb'].unique():
    # Filter data for the specific suburb
    suburb_data = df[df['suburb'] == suburb].set_index('year')
    
    # Dictionary to store the forecasted values for each column
    suburb_forecast = {'suburb': suburb, 'year': forecast_years}
    
    for column in columns_to_forecast:
        # Extract the time series data for the current column
        ts = suburb_data[column]
        
        # Fit the ARIMA model (p, d, q can be optimized or set manually)
        model = ARIMA(ts, order=(1,1,1))
        fitted_model = model.fit()
        
        # Forecast the next 6 years
        forecast = fitted_model.forecast(steps=len(forecast_years))
        
        # Add the forecasted values to the suburb_forecast dictionary
        suburb_forecast[column] = forecast.values
    
    # Append the forecast to the result list
    forecast_results.append(pd.DataFrame(suburb_forecast))

# Concatenate all forecast results into a single DataFrame
forecast_df = pd.concat(forecast_results).reset_index(drop=True)

# Display the forecasted DataFrame
forecast_df.head(20)

In [140]:
# Concatenate df_interpolated and forecast_df along the row axis (axis=0)
df_combined = pd.concat([df_interpolated, forecast_df], axis=0)

# Sort by suburb and year to maintain chronological order
df_combined = df_combined.sort_values(by=['suburb', 'year']).reset_index(drop=True)

In [ ]:
# First, select only the numeric columns
numeric_columns = df_combined.select_dtypes(include=[np.number]).columns

# Replace negative values with zero only in the numeric columns
df_combined[numeric_columns] = df_combined[numeric_columns].applymap(lambda x: 0 if x < 0 else x)

# Replace negative values with zero and round the numbers to integers in the numeric columns
df_combined[numeric_columns] = df_combined[numeric_columns].applymap(lambda x: round(max(0, x)))

# Check the DataFrame to ensure negative values are handled
df_combined.head(30)

In [142]:
df_combined.to_csv('../data/curated/income_forecasted.csv', index=False)